In [1]:
import argparse
import sys
import torch.nn as nn
import torch
from torch.optim import Adam
from torch.utils import data
import time, os, re
from collections import OrderedDict, defaultdict
import collections
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import pickle,gzip
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [ ]:
class Network(nn.Module):
    def __init__(self, input_size, hidden_size1,hidden_size2,hidden_size3,hidden_size4,hidden_size5,hidden_size6, output_size):
        super(Network, self).__init__()
        self.layer1=nn.Sequential(nn.Linear(input_size,hidden_size1),
                                  nn.ReLU(),
                                  nn.Linear(hidden_size1,hidden_size1),
                                  nn.ReLU(),
                                  nn.Linear(hidden_size1,hidden_size1),
                                  nn.ReLU(),
                                  nn.Linear(hidden_size1,hidden_size1),
                                  nn.ReLU(),
                                  nn.Dropout(0.2)
                                  
                                  )
        self.layer2=nn.Sequential(nn.Linear(hidden_size1,hidden_size2),
                                  nn.ReLU(),
                                  nn.Linear(hidden_size2,hidden_size2),
                                  nn.ReLU(),
                                  nn.Linear(hidden_size2,hidden_size2),
                                  nn.ReLU(),
                                  nn.Dropout(0.1)
                                  )
        self.layer3=nn.Sequential(nn.Linear(hidden_size2,hidden_size3),
                                  nn.ReLU(),
                                  nn.Linear(hidden_size3,hidden_size3),
                                  nn.ReLU(),
                                  nn.Linear(hidden_size3,hidden_size3),
                                  nn.ReLU(),
                                  nn.Dropout(0.3)
                                  )
        self.layer4=nn.Sequential(nn.Linear(hidden_size3,hidden_size4),
                                  nn.ReLU(),
                                  nn.Linear(hidden_size4,hidden_size4),
                                  nn.ReLU(),
                                  nn.Linear(hidden_size4,hidden_size4),
                                  nn.ReLU(),
                                  nn.Dropout(0.2)
                                  )
        self.lastlayer=nn.Sequential(nn.Linear(hidden_size4,hidden_size5),
                                  nn.ReLU(),
                                  nn.Linear(hidden_size5,hidden_size5),
                                  nn.ReLU(),
                                  nn.Linear(hidden_size5,hidden_size6),
                                  nn.ReLU(),
                                  nn.Linear(hidden_size6,output_size)
                                  
                                  )
    def forward(self,x):
        out=self.layer1(x)
        out=self.layer2(out)
        out=self.layer3(out)
        out=self.layer4(out)
        out=self.lastlayer(out)
        return out

In [4]:
def Perovskite_tensor():
    with gzip.open('data_p.pickle.gz', 'rb') as ifp:
        dataset=pickle.load(ifp)
    targetvalues=np.zeros(len(dataset))
    normalized_targetvalues=np.zeros(len(dataset))
    Middle_stepA=[]
    Middle_stepB=[]
    Middle_stepC=[]

    for i in range(len(dataset)):
        targetvalues[i]=(float(dataset[i][0]))
        Middle_stepA.append(dataset[i][1])
        Middle_stepB.append(dataset[i][2])
        Middle_stepC.append(dataset[i][3])
        mean=0
        standard_deviation=0

   


    for j in range(len(targetvalues)):
        mean +=targetvalues[j]/len(targetvalues)


    for k in range(len(targetvalues)):
        standard_deviation +=((targetvalues[k]-mean)**2)/len(targetvalues)
    
    standard_deviation=(standard_deviation)**(1/2)


    for s in range(len(dataset)):
        normalized_targetvalues[s]=(targetvalues[s]-mean)/standard_deviation
        


    

    elementsmerged = ['H', 'Li', 'Be', 'B', 'C', 'N', 'O', 'F', 'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'Cl', 'K', 'Ca', 'Sc', 'Ti', 'V', 
            'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'Ga', 'Ge', 'As', 'Se', 'Br', 'Kr', 'Rb', 'Sr', 'Y', 'Zr', 'Nb', 
            'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In', 'Sn', 'Sb', 'Te', 'I', 'Xe', 'Cs', 'Ba', 'La', 'Ce', 'Pr', 
            'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu', 'Hf', 'Ta', 'W', 'Re', 'Os', 'Ir', 
            'Pt', 'Au', 'Hg', 'Tl', 'Pb', 'Bi', 'Ac', 'Th', 'Pa', 'U', 'Np', 'Pu']


    formulare = re.compile(r'([A-Z][a-z]*)(\d*)')
    def parse_formula(formula):
        pairs = formulare.findall(formula)
        length = sum((len(p[0]) + len(p[1]) for p in pairs))
        assert length == len(formula)
        formula_dict = defaultdict(int)
        for el, sub in pairs:
            formula_dict[el] += float(sub) if sub else 1
        return formula_dict

    
    formulasA = [parse_formula(x) for x in Middle_stepA]
    formulasB = [parse_formula(x) for x in Middle_stepB]
    formulasC = [parse_formula(x) for x in Middle_stepC]

    input1 = np.zeros(shape=(len(formulasA), 86), dtype=np.float32)
    input2 = np.zeros(shape=(len(formulasB), 86), dtype=np.float32)
    input3 = np.zeros(shape=(len(formulasC), 86), dtype=np.float32)
    inputmerged = np.zeros(shape=(len(formulasA), 86), dtype=np.float32)
    "To create 3*65 input"
    i = -1
    for formula in formulasA:
        i+=1
        keys = formula.keys()
        for k in keys:
            input1[i][elementsmerged.index(k)] = 1/5
    dataA = input1

    j=-1
    for formula in formulasB:
        j+=1
        keys = formula.keys()
        for k in keys:
            input2[j][elementsmerged.index(k)] = 1/5
    dataB = input2
    n=-1
    for formula in formulasC:
        n+=1
        keys = formula.keys()
        for k in keys:
            input3[n][elementsmerged.index(k)] = 3/5
    dataC = input3
    "To create 65 input"
    for s in range(len(dataC)):
        inputmerged[s]=(dataA[s]+dataB[s]+dataC[s])

    "dataset for merged set"
    X_data=torch.from_numpy(inputmerged)
    Y_data=torch.from_numpy(normalized_targetvalues).float()
    Y_data=Y_data.reshape(-1,1)
    


   
        
    
   
    return(X_data, Y_data)




class Dataset(data.Dataset):
  
  def __init__(self, inputvector, labels):
        
        self.labels = labels
        self.inputvector = inputvector
        
        
  def __len__(self):
        
        return len(self.inputvector)

  def __getitem__(self, index):
      
      X=self.inputvector[index]
      Y=self.labels[index]
      
      return X,Y
the_data=Dataset(Perovskite_tensor()[0],Perovskite_tensor()[1])
train_dataset, test_dataset = torch.utils.data.random_split(the_data, [20000, 228951])


tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.2000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.2000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.6000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000])


In [ ]:
"Create summary writer"
writer = SummaryWriter('/content/drive/My Drive/Colab Notebooks/Tensorboardsave/Perovskite_Elemnet')

"Deciding to run on gpu or cpu"
device = torch.device("cuda" if  torch.cuda.is_available() else "cpu") 

train_loader=torch.utils.data.DataLoader(train_dataset,batch_size=batchsize,shuffle=True)
test_loader=torch.utils.data.DataLoader(test_dataset,batch_size=batchsize,shuffle=False)

"The different layer sizes if the network. Note:some layers share size"
input_size=86
hidden_size1=1024
hidden_size2=512
hidden_size3=256
hidden_size4=128
hidden_size5=64
hidden_size6=32
output_size=1 

"Retrive the model"
model=Network(input_size,hidden_size1,hidden_size2,hidden_size3,hidden_size4,hidden_size5,hidden_size6,output_size).to(device) 



"Create optimizer aswell as training/test loss function "
Testloss=nn.L1Loss()
Trainingloss=nn.MSELoss()
optimizer=torch.optim.Adam(model.parameters(), lr=learningrate)

"Defining variables for the training loop"
best_test_error=100
best_step=0
step=0
patience_steps = int((patience * 20000)/(batchsize))

"Create test function"
def test(epoch): 
    model.eval() #deactivates dropout
    test_loss=0 
    with torch.no_grad(): #no need for gradients in test phase
        k=0
        for i,(inputs,labels) in enumerate(test_loader):
            inputs=inputs.to(device)
            labels=labels.to(device)
            output=model(inputs)
            test_loss +=Testloss(output,labels)
            k +=1
    model.train()        
    return test_loss/k 

"Create training function"
def train(epoch):
    model.train()
    global best_test_error #have to made global
    global step
    global best_step
    k=0
    Totloss=0 #Show accumalted loss, not just over one minibatch
    
    for batch_idx,(inputs,target) in enumerate(train_loader):
        
        inputs=inputs.to(device)
        target=target.to(device)
        optimizer.zero_grad()
        output=model(inputs)
        loss=Trainingloss(output,target)
        loss.backward()
        optimizer.step()
        step +=1
        Totloss +=loss.item()
        k +=1  

   
 
    
    print('Step: {} Train Epoch:{} Training Loss:{}'.format(step,epoch,Totloss/k))
    writer.add_scalar('training_loss',Totloss/k,step)
        
       
            
"Create training loop"
epoch=0
while epoch < (epochs + 1):
    train(epoch)
    if epoch%Evalfreq==0:
       Testsetloss=test(epoch)
       if best_test_error > Testsetloss:
          best_test_error=Testsetloss
          best_step=step
       print('Test set loss: {} Best test loss:{}'.format(Testsetloss,best_test_error))
       writer.add_scalar('test_loss',Testsetloss,step)

    if (best_step + patience_steps) <= step:
           print('No improvement in the last {} steps, best test error acheived was {}'.format(patience_steps,best_test_error))
           print('Done!')
           break
    
    epoch +=1
writer.close()